In [3]:
from pathlib import Path

import numpy as np
import pandas as pd
import jsonpickle

import seaborn as sns
import matplotlib.pyplot as plt

In [20]:
RUNDIR = "../output"
RUNID = "20240130_203925"

In [55]:
config_files = list(Path(RUNDIR).glob(f"{RUNID}*.config.json"))
print(f"Found {len(config_files)} config files")

Found 18 config files


In [56]:
config = jsonpickle.decode(config_files[0].read_text())
data_file = Path(RUNDIR, config_files[0].name.split(".")[0] + ".csv")
data = pd.read_csv(data_file)
data.head()

,round,client,stage,variable,value
0,1,0,train,loss,2.238653
1,1,0,train,n_samples,750.000000
2,1,0,test,loss,2.172284
3,1,0,test,f1,0.188395
4,1,0,test,n_samples,750.000000


In [64]:
configs = {}
dfs = {}
for config_file in config_files:
    config = jsonpickle.decode(config_files[0].read_text())
    data_file = Path(RUNDIR, config_files[0].name.split(".")[0] + ".csv")
    sub_id = config_files[0].name.split(".")[0].split("_")[-1]
    
    data = pd.read_csv(data_file)
    data["id"] = sub_id
    data["algorithm"] = config["algorithm"]
    data["dataset"] = config["dataset"]
    data["imbalance"] = config["imbalance_type"]
    data["imbalance_val"] = config["imbalance_value"]
    
    dfs[sub_id] = data
    configs[sub_id] = config
    
data = pd.concat(dfs.values(), axis=0)

In [65]:
data = data.pivot_table(values = "value", index = ["round", "client", "stage", "id", "algorithm", "dataset", "imbalance", "imbalance_val"], columns = ["variable"]).reset_index()
data.head()

variable,round,client,stage,id,algorithm,dataset,imbalance,imbalance_val,f1,loss,n_samples
0,1,0,test,0,FedAvg,mnist,iid,0.1,0.188395,2.172284,750.0
1,1,0,train,0,FedAvg,mnist,iid,0.1,NaN,2.238653,750.0
2,1,2,test,0,FedAvg,mnist,iid,0.1,0.448013,1.971413,750.0
3,1,2,train,0,FedAvg,mnist,iid,0.1,NaN,2.091077,750.0
4,1,4,test,0,FedAvg,mnist,iid,0.1,0.557664,1.590619,750.0
